In [11]:
# poröse layer und air-layer
import plotly.graph_objs as go
import csv
import numpy as np


In [3]:
# EA-Modell
class EP():
# f frequence
# t Thickness , 
# theta, oblique angle
# density, Materialdichte
# E theYoung’s modulus, 
# Poisson’s ratio
# viscosity. luftvisikosität
## Formulas from pdf Approximation
    def __init__(self, f, t, c0, theta, density, E, v, eta):
        ## Formulas from book Acoustic Waves
        self.f = f
        self.t = t
        self.c0 = c0
        self.theta = np.pi*theta/180
        self.density = density
        self.E = E
        self.v = v 
        self.eta = eta # loss factor
        
        self.omega = 2*np.pi*self.f # angular frequency
        self.k0 = self.omega/self.c0
        self.D = self.E * self.t**3 / (12 * (1 - self.v**2)) # Bending stiffness
        self.m_dot = self.density * self.t
        self.cw = (self.omega**2 * self.D / self.m_dot)**(1/4)
        self.kw = self.omega/self.cw

    # def get_Z(self):
    #
    #     Z = self.D/(1j*self.omega)*((self.k0*np.sin(self.theta))**4 - self.kw**4)
    #
    #     return Z

    def get_fc(self):
        fc_dot = (self.c0**2)/(2*np.pi* np.sin(self.theta)) * np.sqrt(self.m_dot/self.D)
        fc = (self.c0**2)/(2*np.pi) * np.sqrt(self.m_dot/self.D)
        return fc

    def get_Z(self):
        try:
            fc = self.get_fc()
            
            Z = 1j* self.m_dot* self.omega * (1- ((f/fc)**2) * (1+ 1j*self.eta) * np.sin(self.theta)**4)
        except ZeroDivisionError:
            print("wrong")
        return Z

In [12]:
#Alu
density = 850
E = 4.1*10**(9)
v = 0.33
t = 15/1000
c0 = 344
theta = 45

eta = 0.1 #loss factor


c0 = 344 #m/s
density_0 = 1.29  # dichte von Luft-Schicht
Z0 = density_0 * c0

# theta = np.pi*theta/180 # theta nur rad sein können
frequency_min = 10
frequency_max = 5000
frequency_range = np.arange(frequency_min, frequency_max)

ims = np.array([])
res = np.array([])
nf = np.array([])

for f in frequency_range:   
    i = 0
    ep = EP(f, t, c0, theta, density, E, v, eta)
    Z = ep.get_Z()
    Zim = np.imag(Z)
    fc = ep.get_fc()

    ims = np.append(ims, Zim/Z0)
    nf = np.append(nf, f/fc)


fig = go.Figure()
fig.add_trace(go.Scatter(x=nf, y=ims))
# fig.update_xaxes(type="log")
fig.show()


In [5]:
# 4.1*10**(9)
# a = np.array([])
# print(a)
# a = np.append(a, 1)
# a = np.append(a, 2)
# print(a)

In [6]:
# # first resonance frequency of the plate
# def calculate_first_resonance_frequency(E, v, m, a, t):
#     # f1 = (c / 2) * math.sqrt(((m / a) ** 2) + ((n / t) ** 2))
#     try:
#         f = (pi**2 *(1+(a/a)**2)) * ((E*t**3) / (12 * (1 - v**2)) * (a/(m*a**3)))**0.5
#     except ZeroDivisionError:
#         print('Bitte richtige Zahl eingeben!')
#     return f

# # Para. Alu.
# E = 7*(10**10)  # Young's modulus
# v = 0.33    # poission ratio 

# a = 200/1000  # length:m
# t = 20/1000  # thickness：m
# volume = a*a*t
# m = 2700*volume # mass

# # first resonance frequency of the plate
# resonance_frequency = calculate_first_resonance_frequency(E, v, m, a, t)
# print("first resonance frequency of the plate:", resonance_frequency, "Hz")
# masse = m/(a*a)
# area = a*a
# print('Masse:', masse,'\narea:',area)

In [7]:
# np.log10(1000)

# q = np.array([[1.   +0.j  ,       0.+2544.69004941j],
#  [0.   +0.j    ,     1.   +0.j        ]])
# q = np.linalg.det(q)
# l = np.log10(np.abs(q))
# q

In [40]:
## Schräge Inzidenz (Oblique Incidence)
## Umgebungsparemeter 
c0 = 344 #m/s
density_0 = 1.29  # dichte von Luft-Schicht

# Para. Alu.
density = 1.2
E = 7*(10**10)  # Young's modulus
v = 0.33    # poission ratio 
eta = 0.1 #loss factor


# Frequenz Bereich
L1 = float(input('Platte Schicht Dicke(mm):'))
L1 = L1/1000
L2 = float(input('Luft Schicht Dicke(mm):'))
L2 = L2/1000
# theta = float(input('Einfallswinkel(Grad):'))
theta = 0
theta = np.pi*theta/180 # theta nur rad sein können
frequency_min = 10
frequency_max = float(input('max. Frequenz:'))
frequency_range = np.arange(frequency_min, frequency_max)

len = int(frequency_max - frequency_min)
alphas = np.array([]) # Array speichert Absorptionsgrad
TLs = np.array([]) # Array speichert Absorptionsgrad
# Impedanz
Z0 = density_0 * c0
#Z2 = luft_dichte * luft_c  # Luft-Schicht

for f in frequency_range:
    # Z1 = MAA(f, d, a, t1, density_0, viscosity)
    Z1 = EP(f, L1, c0, theta, density, E, v, eta).get_Z()
    if f == frequency_max-1:
        print(Z1)
    Z2 = Z0
    k0 = 2*np.pi*f / c0
    kx = 2 * np.pi * f / c0 * np.sin(theta)
    k2 = k0
    k_z = np.sqrt(k0 ** 2 - kx ** 2)



    # T1, T2, T3 definieren

    T1 = np.array([[1, Z1],
                    [0, 1]])

    #
    # T2 = np.array([[np.cos(k2*L2), 1j*Z2* np.sin(k2*L2)],
    #             [(1j/Z2) * np.sin(k2*L2), np.cos(k2*L2)]])

    T2 = np.array([[np.cos(k_z * L1), 1j * Z2 * (k0 / k_z) * np.sin(k_z * L1)],
          [(1j / Z2) * (k_z / k0) * np.sin(k_z * L1), np.cos(k_z * L1)]])
    # Rigid Backed, kein Einfluss auf T_total
    T3 = np.array([[1, 0],
                [0, 1]])

    # T_total berechnen
    T_total = np.matmul(T1, T2)

    #Absorptionsgrad berechnen
    # R = (Z1 - Z0) / (Z1  + Z0)
    # R = (T_total[0, 0] /T_total[1, 0]- Z0) / (T_total[0, 0]/T_total[1, 0] + Z0) # Reflexionskoeffizient
    R = (T_total[0, 0] * np.cos(theta) - Z0*T_total[1, 0]) / (T_total[0, 0] * np.cos(theta) + Z0*T_total[1, 0]) # Reflexionskoeffizient

    alpha = 1 - (np.abs(R)**2)
    alphas = np.append(alphas, alpha)
  
    T_total = np.linalg.det(T_total)
    TL = -10*np.log10(np.abs(T_total)**2)
    TLs = np.append(TLs, TL)

        # alphas = np.append(alphas[i+1,:], alpha)




/Users/rapha/Library/Caches/pypoetry/virtualenvs/absorb/lib/python3.10/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning:

divide by zero encountered in det

/Users/rapha/Library/Caches/pypoetry/virtualenvs/absorb/lib/python3.10/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning:

invalid value encountered in det



753.9068386378643j


In [41]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=frequency_range, y=alphas))
fig.update_xaxes(type="log")
fig.show()

In [44]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=frequency_range, y=TLs))
fig.update_xaxes(type="log")
fig.show()

In [26]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=frequency_range, y=alphas))
# fig.update_xaxes(type="log")
fig.show()

In [27]:
print(TLs)

[-0.00000000e+00 -0.00000000e+00  9.64327467e-16 ... -5.78596480e-13
 -1.08004676e-13  1.05690290e-12]
